<a href="https://colab.research.google.com/github/dilasko/barbot/blob/main/barbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install pandas gradio google-genai
# Bu komut kütüphaneleri yüklerpandas, gradio ve google-genai
#Pandas veri analizi ve tablo işlemleri için kullanılır,
# Gradio web arayüzleri ile demo uygulamaları oluşturmayı sağlar
#Google-genai ise Google’ın yapay zekâ modellerine erişim imkânı sunar.

In [2]:
import os

# Gemini API anahtarınızı buraya yapıştırıp ortam değişkeni olarak ayarlanmasını sağlar
os.environ['GEMINI_API_KEY'] = 'Kendi anahtarınızı yazın'

In [8]:
import pandas as pd
import gradio as gr
#İçeri atılan pandas ve gradio kütüphanelerine kolay ulaşmak için kısaltmasına atanr.
import random
import os
# Gemini API kütüphaneleri
from google import genai
from google.genai.errors import APIError

# ----------------------------------------------------------------------
# 1. VERİ YÜKLEME VE API İSTEMCİSİ AYARLARI
# ----------------------------------------------------------------------

# Veri setini yüklenir.
try:
    df = pd.read_csv('final_cocktails.csv')
    df['ingredients_lower'] = df['ingredients'].astype(str).str.lower()
    #Bu satır, “ingredients” (malzemeler) sütunundaki tüm verileri:
    #Önce metin (string) tipine dönüştürür → astype(str)
    #Sonra küçük harfe çevirir → .str.lower()
    #Son olarak bu dönüştürülmüş veriyi ingredients_lower adlı yeni bir sütuna ekler.
    #Yani büyük/küçük harf farkını ortadan kaldırır, aramalarda kolaylık sağlar.
except FileNotFoundError:
  #Bu kısım, yukarıdaki read_csv işlemi sırasında dosya bulunmazsa devreye girer.
  #Yani Python, “final_cocktails.csv diye bir dosya yok” hatası aldığında bu bloğu çalıştırır
    print("HATA: 'final_cocktails.csv' dosyası bulunamadı. Lütfen dosyayı Colab'e yüklediğinizden emin olun.")
    df = pd.DataFrame()

# Gemini API istemcisini başlatma
GEMINI_ENABLED = False
client = None
try:
    api_key = os.environ.get('GEMINI_API_KEY')
    if api_key:
        client = genai.Client(api_key=api_key)
        GEMINI_ENABLED = True
        print("✅ Gemini API istemcisi başarıyla başlatıldı. Tüm çıktı alanları Türkçe'ye çevrilecektir.")
    else:
        print("❌ Gemini API Anahtarı ortam değişkeninde bulunamadı. Çeviri ve Görsel oluşturma devre dışı.")
except Exception as e:
    print(f"❌ Gemini API istemcisi başlatılamadı: {e}")

# ----------------------------------------------------------------------
# 2. GEMINI ÇEVİRİ FONKSİYONLARI
# ----------------------------------------------------------------------

def safe_translate(prompt, model='gemini-2.5-flash'):
    """Genel, güvenli çeviri işlevi."""
    #veri setlerini türkçeye çevirmek için çeviri fonksiyonu
    if not GEMINI_ENABLED:
        return None
    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        return response.text.strip()
    except Exception as e:
        # Hata durumunda None döndür
        return None

def translate_item(item_en, context_prompt):
    """Kısa, tek bir terimi çevirir."""
    result = safe_translate(
        prompt=f"Bu İngilizce terimi Türkçe'ye çevir: '{item_en}'. Çeviri yaparken sadece çevrilmiş terimi döndür. Bağlam: {context_prompt}"
    )
    # Çeviri başarısız olursa orijinali döndür
    return result if result else item_en

def translate_instructions(text_en):
    """Kokteyl talimatlarını profesyonel bir dille çevirir."""
    result = safe_translate(
        prompt=f"Bu kokteyl talimatını profesyonel bir barmen edasıyla ve doğal bir dille Türkçe'ye çevir: {text_en}"
    )
    return result if result else f"**(Çeviri yapılamadı - Orijinal Metin)**:\n{text_en}"

def translate_ingredient_list(list_en):
    """Malzeme listesinin tamamını çevirir (daha iyi bağlam için)."""
    result = safe_translate(
        prompt=f"Aşağıdaki kokteyl malzeme listesini ve ölçülerini tam ve doğru Türkçe'ye çevir. Formatı (madde işaretleri) aynen koru. Sadece çevrilmiş listeyi döndür:\n{list_en}"
    )
    return result if result else f"**(Çeviri yapılamadı - Orijinal Liste)**:\n{list_en}"


# ----------------------------------------------------------------------
# 3. ANA CHATBOT FONKSİYONU
# ----------------------------------------------------------------------

def kokteyl_tarifi_al(malzeme_girisi):
    """
    Kullanıcının girdiği malzemelerden en az birini içeren kokteyl tarifini döndürür.
    """
    if df.empty:
        return "Veri seti yüklenemedi. Lütfen dosyanın adını kontrol edin.", None

    # 1. GİRDİYİ İŞLEME: Virgülle ayırma ve temizleme
    search_terms = [term.strip().lower() for term in malzeme_girisi.split(',') if term.strip()]

    if not search_terms:
        output_text = "⚠️ Lütfen bir malzeme ismi veya virgülle ayrılmış birden fazla malzeme giriniz."
        return output_text, None

    # 2. FİLTRE OLUŞTURMA: OR (veya) mantığı ile filtreleme
    filter_condition = False
    for term in search_terms:
        # Her bir terim için `str.contains` sonucunu mevcut koşulla birleştir (OR)
        filter_condition = filter_condition | df['ingredients_lower'].str.contains(term, na=False)

    # 3. FİLTREYİ UYGULAMA
    filtered_df = df[filter_condition]

    if filtered_df.empty:
        # Kullanıcının girdiği tüm terimleri birleştirip gösterme
        terimler = ", ".join([f"'{t}'" for t in search_terms])
        output_text = f"❌ Girdiğiniz malzemelerden (**{terimler}**) herhangi birini içeren bir kokteyl tarifi bulunamadı. Başka malzemeler deneyin."
        return output_text, None

    # Birden fazla sonuç varsa, rastgele birini seç
    selected_recipe = filtered_df.sample(n=1).iloc[0]

    name = selected_recipe['name']
    alcoholic_en = selected_recipe['alcoholic']
    category_en = selected_recipe['category']
    glass_en = selected_recipe['glassType']
    instructions_en = selected_recipe['instructions']
    ingredients_en = selected_recipe['ingredients'].strip("[]").replace("'", "")
    measures_en = selected_recipe['ingredientMeasures'].strip("[]").replace("'", "")
    thumbnail_url = selected_recipe['drinkThumbnail']

    # 4. TÜM ALANLARI ÇEVİRME

    # Alkollü/Alkolsüz kontrolü
    if alcoholic_en == 'Alcoholic':
        alcoholic_tr = "Evet (Alkollü)"
    elif alcoholic_en == 'Non Alcoholic':
        alcoholic_tr = "Hayır (Alkolsüz)"
    else:
        alcoholic_tr = translate_item(alcoholic_en, "Alkollü olup olmadığını belirten bir durum.")

    category_tr = translate_item(category_en, "Kokteylin ait olduğu içecek kategorisi.")
    glass_tr = translate_item(glass_en, "Kokteylin servis edildiği bardak tipi.")

    # Malzeme Listesini Formatlama ve Çevirme
    try:
        ingredient_list_en = [
            f"• {m.strip()} {i.strip()}"
            for m, i in zip(measures_en.split(','), ingredients_en.split(','))
        ]
        ingredients_formatted_en = "\n".join(ingredient_list_en)
    except Exception:
        ingredients_formatted_en = f"Malzemeler: {ingredients_en}\nÖlçüler: {measures_en}"

    ingredients_formatted_tr = translate_ingredient_list(ingredients_formatted_en)
    instructions_tr = translate_instructions(instructions_en)

    # SONUCU TÜRKÇE OLARAK BİÇİMLENDİRME
    output_text = f"""
    ✨ **Kokteyl Adı:** {name}
    ---
    🍸 **Kategori:** {category_tr}
    🧊 **Bardak Tipi:** {glass_tr}
    🥂 **Alkollü:** {alcoholic_tr}
    ---
    **🧪 Malzemeler ve Ölçüler:**
    {ingredients_formatted_tr}
    ---
    **📜 Yapılış (Türkçe Talimatlar):**
    {instructions_tr}
    """

    return output_text.strip(), thumbnail_url

# ----------------------------------------------------------------------
# 4. GRADIO BLOCKS ARAYÜZÜ (Düzenli Çıktı)
# ----------------------------------------------------------------------

with gr.Blocks(title="🍸 Kokteyl Tarif Chatbot'u B-A-R*B*O*T") as demo:
    gr.Markdown(
        """
        # 🍸 Kokteyl Tarif Chatbot'u B-A-R*BOT
        Elindeki bir veya birden fazla malzemeyi virgülle ayırarak gir, sistem sana o malzemelerden en az birini içeren rastgele bir kokteyl tarifi sunsun!

        """
    )

    # Giriş ve Butonlar
    with gr.Row():
        ingredient_input = gr.Textbox(
            lines=1,
            placeholder="Virgülle ayırarak birden fazla malzeme girin (Örn: Gin, Lime, Sugar)",
            label="Hangi malzemelerle kokteyl tarifi istersiniz?"
        )
        submit_btn = gr.Button("Tarifi Bul", variant="primary", scale=0)

    # Çıktılar
    with gr.Row():
        recipe_output = gr.Textbox(
            label="Kokteyl Tarifi",
            scale=2,
            lines=25
        )

        image_output = gr.Image(
            label="Kokteyl Görseli",
            type="pil",
            scale=1
        )

    # Butonun eylemini tanımlama
    submit_btn.click(
        fn=kokteyl_tarifi_al,
        inputs=ingredient_input,
        outputs=[recipe_output, image_output]
    )

    # Örnekler (Çoklu giriş örnekleri eklendi)
    gr.Examples(
        examples=[
            ["Vodka"],
            ["Rum, Lime"],
            ["Whiskey, Sugar, Bitters"],
        ],
        inputs=ingredient_input
    )

# Colab'de çalıştırmak için launch() metodunu kullanın
demo.launch(share=True)

✅ Gemini API istemcisi başarıyla başlatıldı. Tüm çıktı alanları Türkçe'ye çevrilecektir.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d97d9f3ea96ec3969.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
